# How long has each report been running?

**We are still considering active reports!**

How long has an active report been running? Is it possible to retrieve the date of its first execution? What is the average number of executions of a report since its creation? Are scheduled reports run more frequently? Which reports took the most time to run?


## What are the most frequent reports?